<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/3_submit_test/gp-submit_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Submit Test

제출 형태도 정해져있으므로 Validation Set을 이용해서 제출해보기

## GCS(Google Cloud Stoarge)에서 Google Drive로 데이터셋 다운받기

google drive를 먼저 colab에 연동한 뒤, 해당 디렉토리에 GCS로부터 데이터셋 다운받기

In [3]:
# GCS에 접근하기 위해 인증

from google.colab import auth
auth.authenticate_user()

In [2]:
# GCS 데이터 용량 확인. 전체를 다운받기에는 구글 클라우드로 감당이 안될듯.
!gsutil ls -l "gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/" | xargs -I{} gsutil du -sh  {}

0 B          gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/lidar
645.92 GiB   gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario
1.08 TiB     gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/tf_example


In [3]:
# 라이다 데이터 validation set만 확인
!gsutil du -ch -s "gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/lidar/validation"

200.35 GiB   gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/lidar/validation
200.35 GiB   total


In [4]:
# 시나리오 데이터 validation set만 확인
!gsutil du -ch -s "gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/validation"

38.4 GiB     gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/validation
38.4 GiB     total


데이터셋을 저장할 Google Drive를 마운트

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# 구글 드라이브 위치. 마운트 잘 됐는지 확인.
GDRIVE_WOD_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed"

!ls -al {GDRIVE_WOD_PATH}

total 12
drwx------ 2 root root 4096 Apr 14 03:28 .ipynb_checkpoints
drwx------ 3 root root 4096 Apr 14 00:03 lidar
drwx------ 2 root root 4096 Mar 29 06:17 tf_example


GCS로부터 데이터셋을 다운로드. 라이다 데이터도 다운받으려고 했는데 몇시간 기다리다가 지쳐서 일단 보류. 당장 필요한 시나리오 데이터만 우선 받아본다

In [7]:
# 시나리오 데이터 다운로드

%%time
%%capture

!mkdir {GDRIVE_WOD_PATH}/scenario

!gsutil -m cp -r \
  "gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/validation" \
  {GDRIVE_WOD_PATH}/scenario/

CPU times: user 6.6 s, sys: 1.27 s, total: 7.86 s
Wall time: 12min 1s


In [8]:
from google.colab import drive
drive.flush_and_unmount()

## 환경세팅

In [9]:
%%capture

!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1 # 최신 버전으로 한 번 해볼까나?

In [13]:
from google.colab import drive
drive.mount('/content/drive')

# 마운트 확인
GDRIVE_WOD_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed"
VALIDSET_PATH   = GDRIVE_WOD_PATH + "/scenario/validation"

!ls -al {VALIDSET_PATH} | wc -l

Mounted at /content/drive
151


## 데이터셋 로드

In [14]:
import glob

# 로컬 디렉토리에 저장된 TFRecord 파일들의 경로 리스트 생성
file_paths = glob.glob(VALIDSET_PATH + "/*.tfrecord*")
len(file_paths)

150

In [15]:
import tensorflow as tf

# TFRecordDataset 클래스를 사용하여 데이터를 읽어들임
dataset = tf.data.TFRecordDataset(file_paths, compression_type='')

In [21]:
# 데이터셋 한 개만 읽어보기
from waymo_open_dataset.protos import scenario_pb2

for data in dataset:
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(bytearray(data.numpy()))
  break

scenario.scenario_id

'a3cfb7f093ccd29'

In [20]:
# tutorial/3_submit_test/gp-submit_test.ipynb